In [24]:
import os
import pandas as pd

# Code to combine DAMASMCPC price data from 2022 to 2025

In [26]:
def fix_hour_ending(row):
    if row["Hour Ending"] == "24:00":
        return row["Delivery Date"] + pd.Timedelta(days=1)
    else:
        time = pd.to_datetime(row["Hour Ending"], format="%H:%M").time()
        return pd.Timestamp.combine(row["Delivery Date"], time)

In [27]:
def process_single_file(filepath):
    df = pd.read_csv(filepath)
    df.columns = df.columns.str.strip()
    df["Delivery Date"] = pd.to_datetime(df["Delivery Date"], format="%m/%d/%Y")
    df["datetime_col"] = df.apply(fix_hour_ending, axis=1)
    df.drop(columns=["Delivery Date", "Hour Ending", "Repeated Hour Flag"], inplace=True)

    return df

In [28]:
years = [2022, 2023, 2024, 2025]
base_path = "./data/prices/raw/"
files = [os.path.join(base_path, f"DAMASMCPC_{year}.csv") for year in years]

all_dfs = [process_single_file(file) for file in files]
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.sort_values("datetime_col").reset_index(drop=True)

# Reorder columns
reordered_cols = ['datetime_col'] + [col for col in combined_df.columns if col != 'datetime_col']
combined_df = combined_df[reordered_cols]

# Save to new file
output_path = "./data/prices/DAMASMCPC_2022_to_2025.csv"
os.makedirs(os.path.dirname(output_path), exist_ok=True)
combined_df.to_csv(output_path, index=False)

print(f"✅ Saved cleaned and combined dataset to: {output_path}")

✅ Saved cleaned and combined dataset to: ./data/prices/DAMASMCPC_2022_to_2025.csv


# Code to combine DAMLZHBSPP price data from 2022 to 2025

In [3]:
def fix_hour_ending(row):
    if row["Hour Ending"] == "24:00":
        return pd.to_datetime(row["Delivery Date"], format="%m/%d/%Y") + pd.Timedelta(days=1)
    else:
        time = pd.to_datetime(row["Hour Ending"], format="%H:%M").time()
        date = pd.to_datetime(row["Delivery Date"], format="%m/%d/%Y")
        return pd.Timestamp.combine(date, time)

In [12]:
def process_single_excel(xlsx_path: str) -> pd.DataFrame:
    # Load Excel file
    df = pd.read_excel(xlsx_path)
    df.columns = df.columns.str.strip()

    # Create datetime column
    df["datetime_col"] = df.apply(fix_hour_ending, axis=1)

    # Pivot SPPs into columns
    pivot_df = df.pivot_table(
        index="datetime_col",
        columns="Settlement Point",
        values="Settlement Point Price"
    ).reset_index()

    return pivot_df

In [13]:
years = [2022, 2023, 2024, 2025]
input_dir = "./data/prices/raw/"
output_path = "data/prices/DAMLZHBSPP_2022_to_2025.csv"

all_dfs = []
for year in years:
    file_path = os.path.join(input_dir, f"DAMLZHBSPP_{year}.xlsx")
    print(f"📂 Processing: {file_path}")
    yearly_df = process_single_excel(file_path)
    all_dfs.append(yearly_df)

# concatenate and sort chronologically
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.sort_values("datetime_col").reset_index(drop=True)

# save to CSV
os.makedirs(os.path.dirname(output_path), exist_ok=True)
combined_df.to_csv(output_path, index=False)
print(f"\n✅ Saved combined file to: {output_path}")

📂 Processing: ./data/prices/raw/DAMLZHBSPP_2022.xlsx
📂 Processing: ./data/prices/raw/DAMLZHBSPP_2023.xlsx
📂 Processing: ./data/prices/raw/DAMLZHBSPP_2024.xlsx
📂 Processing: ./data/prices/raw/DAMLZHBSPP_2025.xlsx

✅ Saved combined file to: data/prices/DAMLZHBSPP_2022_to_2025.csv


# Code to combine RTMLZHBSPP price data from 2022 to 2025

In [17]:
def process_15min_excel_file(xlsx_path: str) -> pd.DataFrame:
    df = pd.read_excel(xlsx_path)
    df.columns = df.columns.str.strip()
    df["Delivery Date"] = pd.to_datetime(df["Delivery Date"], format="%m/%d/%Y")

    # Map Delivery Interval to minutes
    interval_to_minute = {1: 0, 2: 15, 3: 30, 4: 45}
    df["minute"] = df["Delivery Interval"].map(interval_to_minute)

    # Compute datetime_col from Delivery Date, Hour, and Interval
    df["datetime_col"] = df.apply(
        lambda row: row["Delivery Date"] + pd.Timedelta(hours=row["Delivery Hour"] - 1, minutes=row["minute"]),
        axis=1
    )

    # Pivot: Each Settlement Point Name becomes its own column
    pivot_df = df.pivot_table(
        index="datetime_col",
        columns="Settlement Point Name",
        values="Settlement Point Price"
    ).reset_index()

    return pivot_df

In [18]:
years = [2022, 2023, 2024, 2025]
input_dir = "./data/prices/raw/"
output_path = "./data/prices/RTMLZHBSPP_2022_to_2025.csv"

all_dfs = []
for year in years:
    file_path = os.path.join(input_dir, f"RTMLZHBSPP_{year}.xlsx")
    print(f"📂 Processing: {file_path}")
    yearly_df = process_15min_excel_file(file_path)
    all_dfs.append(yearly_df)

# Combine and sort chronologically
combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df = combined_df.sort_values("datetime_col").reset_index(drop=True)

# Save to CSV
os.makedirs(os.path.dirname(output_path), exist_ok=True)
combined_df.to_csv(output_path, index=False)
print(f"\n✅ Saved combined 15-min interval file to: {output_path}")

📂 Processing: ./data/prices/raw/RTMLZHBSPP_2022.xlsx
📂 Processing: ./data/prices/raw/RTMLZHBSPP_2023.xlsx
📂 Processing: ./data/prices/raw/RTMLZHBSPP_2024.xlsx
📂 Processing: ./data/prices/raw/RTMLZHBSPP_2025.xlsx

✅ Saved combined 15-min interval file to: ./data/prices/RTMLZHBSPP_2022_to_2025.csv
